## Recipe to carve out dimension tables from a "fat" flat table

The idea is that we have a flat table with lots of lookup values already brought in.
While that is very convenient sometimes, especially when using it for a pivot table in Excel, it is not such a good idea if you want to analyse in PowerBI

In [2]:
import pandas as pd
import numpy as np

In [3]:
# We load the demo data and cleanup the headers for easier typing

df=pd.read_excel("./test_data/demo_data.xlsx",sheet_name="demo_claims")
df.columns=df.columns.str.lower().str.replace(" ","_")
df.columns

Index(['memberid', 'claimid', 'providerid', 'providername', 'provider_address',
       'providercountry', 'benefitid', 'benefitdesc', 'procedurecode',
       'treatmentcountry', 'amount', 'currencycode', 'exchrategbp',
       'amountgbp'],
      dtype='object')

We do a few calculations to profile the dataframe.
These are not used by the main routine (yet) but tells you the cardinality of the values, ie how many unique values there are in each field and can suggest which ones are worth moving to categorical dimensions. 


In [12]:
count_unique= df.nunique().to_list()
count_nulls=df.isnull().sum().to_list()
card_perc= [i[0]/i[1] for i in zip(count_unique, df.count())] #zip just joins similar lists together  
cardinality=list(zip(df.columns,count_unique, df.count(),count_nulls,card_perc))
cardinality_df=pd.DataFrame(cardinality, columns=["name","cardinality","count","count_nulls","card_percent"])
cardinality_df

,name,cardinality,count,count_nulls,card_percent
0,memberid,5,17,0,0.294118
1,claimid,8,17,0,0.470588
2,providerid,5,17,0,0.294118
3,providername,5,17,0,0.294118
4,provider_address,4,16,1,0.250000
5,providercountry,2,17,0,0.117647
6,benefitid,8,17,0,0.470588
7,benefitdesc,8,17,0,0.470588
8,procedurecode,8,13,4,0.615385
9,treatmentcountry,2,17,0,0.117647


Now we do the real work

The idea is that we take a key and suggested attributes and we do what we would do manually.
First we see if the attributes are 1:1 (or zero) with the key, ie one value of the key doesnt have two or more values of the attributes in the dataset. If it passes that test it gets added to the list of attributes to carve out.

Then once all is checked we create a separate dataframe with the unique keys/attributes and for a better "joining experience", we create a surrogate key with a simple integer in both the original fact table and the new lookup/dimension table. 
Actually we dont change the original dataframe but we create a copy of it, having removed the key+attributes and added the surrogate key. The new keys would have same name, so they should be easy to auto-join by PowerBI etc.


In [5]:
def dim_from_flat(df, key_name, attributes,flag_ignore_blanks=False, flag_force_multiple_values=False):
    dim_fields=[key_name]
    dim_post_merge_fields=[]
    dim_groupby_fields=[]
    for i in attributes:
        df_unique=df[[key_name,i]].drop_duplicates(ignore_index=True)
        # we check that picking that attribute doesnt create duplicate keys, which 
        # would be inconsistent with a one-to-many situation. 
        df_check_key_unique=df_unique[df_unique[key_name].duplicated(keep=False)]
        if len(df_check_key_unique)==0:
            print(i," check ok, only unique values")
            dim_fields.append(i)
        else:
            if len(df_check_key_unique)==2 and (pd.isnull(df_check_key_unique[i])).sum()==1 and flag_ignore_blanks==True:
                print(i," has mix of blank and unique value, will ignore and include afterwards")
                print(df_check_key_unique.head())
                dim_post_merge_fields.append(i)
            else:    
                if flag_force_multiple_values==True:
                    print(i," has multiple values, we are going to pick the first one")
                    print(df_check_key_unique.head())
                    dim_groupby_fields.append (i)
                else:
                    print(i," skipped, found multiple/blank values per key ", key_name)
                    print(df_check_key_unique.head())
                
    df_dim=df[dim_fields].drop_duplicates(ignore_index=True).sort_values([key_name])
    for i in dim_post_merge_fields:
        print("Post merging blank values",i)
        df_temp=df[[key_name,i]][pd.notnull(df[i])].drop_duplicates(ignore_index=True)
        print(df_temp.head())
        df_dim=df_dim.merge(df_temp,how='left',left_on=key_name, right_on=key_name)
    for i in dim_groupby_fields:
        print("Grouping and adding duplicated values for ",i," picking the first one")
        df_temp=df[[key_name,i]][pd.notnull(df[i])].drop_duplicates(ignore_index=True)
        df_temp=df_temp.groupby(key_name).agg(key=(i,"first"),counts=(i,'count')).reset_index()
        df_temp.rename(columns = {'key':i}, inplace = True)
        df_temp.rename(columns = {'counts':i+"_counts"}, inplace = True)
        df_dim=df_dim.merge(df_temp,how='left',left_on=key_name, right_on=key_name)
    df_final_check_key_unique=df_dim[df_dim[key_name].duplicated(keep=False)]
    if len(df_final_check_key_unique)==0:
        print("Final check successful, ", key_name," dimension table has unique values." )
    else:
        print("Final check failed, table ", key_name," still have duplicates, aborting")
        print(df_final_check_key_unique.head())
        return none,none
    df_dim.insert(0, "dimkey_"+key_name, range(1, 1 + len(df_dim)))
    df_dim.reset_index(drop=True,inplace=True)
    df_fact=df.merge(df_dim[[key_name,"dimkey_"+key_name]],how='left',left_on=key_name, right_on=key_name)
    df_fact.drop(dim_fields, axis=1, inplace=True)
    if flag_ignore_blanks:
        df_fact.drop(dim_post_merge_fields, axis=1, inplace=True)
    if flag_force_multiple_values:
        #This is probably not a good idea, as we are losing information of the aggregation done
        #In real life you probably want to keep the original column in the fact table
        df_fact.drop(dim_groupby_fields, axis=1, inplace=True)
    print("Process completed successfully")
    return df_dim, df_fact



In [6]:
df_cur,df_fact= dim_from_flat(df,"currencycode",["exchrategbp",'claimid'],True,False)
df_cur

exchrategbp  has mix of blank and unique value, will ignore and include afterwards
  currencycode  exchrategbp
1          GBP          1.0
2          GBP          NaN
claimid  skipped, found multiple/blank values per key  currencycode
  currencycode claimid
0          USD     C01
1          USD     C02
2          GBP     C02
3          GBP     C03
4          USD     C03
Post merging blank values exchrategbp
  currencycode  exchrategbp
0          USD     1.276733
1          GBP     1.000000
Final check successful,  currencycode  dimension table has unique values.
Process completed successfully


,dimkey_currencycode,currencycode,exchrategbp
0,1,GBP,1.000000
1,2,USD,1.276733


In [7]:
df_prov,df_fact=dim_from_flat(df_fact,"providerid",['providername','provider_address','providercountry'],True,True)
df_prov

providername  check ok, only unique values
provider_address  has mix of blank and unique value, will ignore and include afterwards
  providerid provider_address
2        P03     3 Privet Dr.
3        P03              NaN
providercountry  has multiple values, we are going to pick the first one
  providerid providercountry
0        P01              US
3        P01              UK
Post merging blank values provider_address
  providerid      provider_address
0        P01     San Francisco, CL
1        P02          3 Privet Dr.
2        P03          3 Privet Dr.
3        P04  1 Kensington Gardens
4        P05     The Shard, London
Grouping and adding duplicated values for  providercountry  picking the first one
Final check successful,  providerid  dimension table has unique values.
Process completed successfully


,dimkey_providerid,providerid,providername,provider_address,providercountry,providercountry_counts
0,1,P01,Clinic Mayo,"San Francisco, CL",US,2
1,2,P02,Pharmacy Excel,3 Privet Dr.,UK,1
2,3,P03,Dr Grim,3 Privet Dr.,UK,1
3,4,P04,Dr Hayes and Asoc.,1 Kensington Gardens,UK,1
4,5,P05,HCA Clinic,"The Shard, London",UK,1


In [10]:
df_proc,df_fact=dim_from_flat(df_fact,"benefitid",["benefitdesc"])
df_proc

benefitdesc  check ok, only unique values
Final check successful,  benefitid  dimension table has unique values.
Process completed successfully


,dimkey_benefitid,benefitid,benefitdesc
0,1,10,Pregnancy Checks
1,2,20,Pain relief/anti inflammatory
2,3,22,Antibiotics
3,4,30,GP Diagnosis
4,5,40,Cancer Treatment
5,6,50,Radiology
6,7,60,CAT Scans
7,8,80,Blood tests


In [11]:
df_fact

,memberid,claimid,procedurecode,treatmentcountry,amount,amountgbp,dimkey_currencycode,dimkey_providerid,dimkey_benefitid
0,M01,C01,901.0,US,1000,783.249121,2,1,1
1,M01,C02,902.0,US,800,626.599297,2,1,1
2,M01,C02,NaN,UK,400,400.000000,1,2,2
3,M02,C03,501.0,UK,200,200.000000,1,3,4
4,M02,C03,901.0,US,1300,1018.223857,2,1,1
5,M02,C03,NaN,UK,300,300.000000,1,2,2
6,M03,C04,NaN,UK,20,20.000000,1,2,3
7,M03,C04,501.0,UK,100,100.000000,1,3,4
8,M04,C05,903.0,US,200,156.649824,2,1,1
9,M04,C06,902.0,US,600,469.949473,2,1,1
